In [1]:
import numpy as np
import sklearn
import sklearn.ensemble
%load_ext autoreload
%autoreload 2

In [2]:
data = np.genfromtxt('../datasets/titanic.txt', delimiter=',')
y_train = data[:, -1]
X_train = data[:, :-1]


In [3]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from Anchor.anchor import Anchor, Tasktype

c = sklearn.ensemble.RandomForestClassifier(n_estimators=100, n_jobs=5, random_state=123)
c.fit(X_train, y_train)
print('Train', sklearn.metrics.accuracy_score(y_train, c.predict(X_train)))

explainer = Anchor(Tasktype.TABULAR)

task_paras = {"dataset": X_train, "column_names": ["PcClass", "Name", "Sex", "Age", "SibSp", "Parch", "Ticket", "Fare", "Cabin", "Embarked"]}
method_paras = {"beam_size": 1, "desired_confidence": 1.0}
anchor = explainer.explain_instance(
    input=X_train[759].reshape(1, -1),
    predict_fn=c.predict,
    method="beam",
    task_specific=task_paras,
    method_specific=method_paras,
    num_coverage_samples=100,
)

INFO:root: Start Sampling
INFO:root: Start Beam Search


Train 1.0


In [4]:
anchor

AnchorCandidate(feature_mask=[2, 0], precision=1.0, n_samples=160, positive_samples=160, coverage=0.54)

In [5]:
visu = explainer.visualize(anchor, X_train[759])
print(visu)

PcClass = 1.0 AND Sex = 1.0


iAnchor expects the tabular dataset to be discretized. This can be done with the lime package as can be seen in the hyperparam notebook. When using said package one could easily convert the discretized back to the undiscretized values (see [lime.discretize module](https://lime-ml.readthedocs.io/en/latest/lime.html)).